# Cross Validation

- 교차검증 : 학습 단계에서 일반화가 더욱 잘 수행되기 위해, 학습 데이터를 여러 단계로 나누어 교차로 검증하는 기법
- Train Set 내에서 Train Set과 Validation Set을 구분하여, 반본적으로 Validation Set 바꿔 가며 학습
- K-Fold Cross Validation : 특정 K 개수 만큼 학습 데이터를 나누어 교차로 검증 (일반적으로 5~10)
- **Stratified K-Fold Cross Validation** : 특정 K 개수 만큼 학습 데이터를 나누는데, 목표변수와 항목간의 비율을 유지한 채로 나누느어 교차로 검증

In [36]:
import pandas as pd
import scipy.stats as stats
from sklearn.pipeline import make_pipeline # 특성공학 + 학습수행
from sklearn.impute import SimpleImputer # 결측값을 처리 (단순대치)
from sklearn.tree import DecisionTreeClassifier # 분류 학습을 수행
from sklearn.model_selection import train_test_split # 학습데이터와 검증데이터 분할
import pickle

In [41]:
df1 = pd.read_csv(r'C:\Users\UserK\Desktop\Ranee\data\ML\12_Data.csv') 
print(df1)
df1.head(2)

     Image ID Diagnosis  Mean Radius  Mean Perimeter  Mean Area  Mean Texture  \
0      842302         M        17.99          122.80     1001.0         10.38   
1      842517         M        20.57          132.90     1326.0         17.77   
2    84300903         M        19.69          130.00     1203.0         21.25   
3    84348301         M        11.42           77.58      386.1         20.38   
4    84358402         M        20.29          135.10     1297.0         14.34   
..        ...       ...          ...             ...        ...           ...   
564    926424         M        21.56          142.00     1479.0         22.39   
565    926682         M        20.13          131.20     1261.0         28.25   
566    926954         M        16.60          108.30      858.1         28.08   
567    927241         M        20.60          140.10     1265.0         29.33   
568     92751         B         7.76           47.92      181.0         24.54   

     Mean Smoothness  Mean 

,Image ID,Diagnosis,Mean Radius,Mean Perimeter,Mean Area,Mean Texture,Mean Smoothness,Mean Compactness,Mean Concavity,Mean Concave Points,...,SE Radius,SE Perimeter,SE Area,SE Texture,SE Smoothness,SE Compactness,SE Concavity,SE Concave Points,SE Symmetry,SE Fractal Dim
0,842302,M,17.99,122.8,1001.0,10.38,0.12,0.27760,0.3001,0.1471,...,1.0950,8.589,153.40,0.9053,0.0064,0.0490,0.0537,0.0159,0.0300,0.0062
1,842517,M,20.57,132.9,1326.0,17.77,0.08,0.07864,0.0869,0.0702,...,0.5435,3.398,74.08,0.7339,0.0052,0.0131,0.0186,0.0134,0.0139,0.0035


- 세포의 기하구조를 입력했을 때, 암/정상 세포를 판단하는 분류모델을 만들고자 한다. 'Mean'이름이 포함된 모든 Column을 X로, Diagnosis를 Y로 선언하여, X를 입력했을 때, Y를 분류하는 분류모델을 생성하시오.
  - 알고리즘은 Decision Tree를 이용
  - 학습 / 검증 데이터 비율은 75% : "25% (Random State - 1234)
  - 생성된 모델을 model_cell.sav로 저장 저장

In [42]:
def func() :
    df3 = df1.filter(like = 'Mean')
    mean_list = []
    for i in df3.columns :
        if stats.normaltest(df3[i])[1] < 0.05 :
            mean_list.append(i)
    return mean_list

In [43]:
list_mean = func()

In [44]:
X = df1[list_mean]
Y = df1['Diagnosis']

In [53]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,random_state=1234,test_size=0.25)

In [54]:
pipe_model = make_pipeline(SimpleImputer(strategy='median'), StandardScaler(), DecisionTreeClassifier())

In [71]:
from sklearn.model_selection import GridSearchCV, cross_validate # CV : Cross Validation

In [72]:
# 수업 외
cross_validate(pipe_model,X_train,Y_train)

{'fit_time': array([0.00645685, 0.00414824, 0.00403428, 0.00439024, 0.0040009 ]),
 'score_time': array([0.00100017, 0.00176001, 0.0019989 , 0.00099945, 0.00177503]),
 'test_score': array([0.90697674, 0.94117647, 0.89411765, 0.95294118, 0.91764706])}

In [56]:
grid_model = GridSearchCV(pipe_model, param_grid={}, cv=5) 
grid_model.fit(X_train, Y_train) # 학습모델은 5개가 나옴
cross_validate
# 학습에 Test Set은 들어가지 않음

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('simpleimputer',
                                        SimpleImputer(strategy='median')),
                                       ('standardscaler', StandardScaler()),
                                       ('decisiontreeclassifier',
                                        DecisionTreeClassifier())]),
             param_grid={})

In [57]:
# 성능이 가장 좋은 모델 찾기 (cross validation에서 기본 성능 지표 -> Acc(분류) / R²(결정계수))
best_model = grid_model.best_estimator_

# grid_model2 = GridSearchCV(pipe_model, param_grid={}, cv=5, scoring='f1') 
# CV= : 몇 개의 Fold를 지정할 것인가 / scoring= : 어떤 자료로 평가할 것인가)

In [19]:
best_model

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('standardscaler', StandardScaler()),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

# Hyper Parameter Tuning 

- Hyper Parameter Tuning : Model을 학습 시키기 전, 학습 알고리즘 내 세팅되어 있는 매개변수
- 학습 알고리즘 내 지정된 매개변수를 사용자가 조절하여, Overfitting을 방지, 학습/일반화 성능을 높이는 기법 (알고리즘 내 존재하는 고유의 Hyper Parameter를 사전에 알고 있어야함)
  - Random Search : 매개변수의 조합을 임의로 설정하여 학습
  - Grid Search : 사용자가 지정한 범위 내 매개변수의 모든 조합에 대한 학습
  - Bayesian Optimization : 베이지안 최적화 기법을 이용해, 불확실성이 가장 낮은 매개변수를 선택해 학습

In [63]:
hypermeter = {'decisiontreeclassifier__max_depth' : range(5,20), # 15가지 경우의 수
             'decisiontreeclassifier__min_samples_split':range(5,10) } # 5가지 경우의 수


grid_model3 = GridSearchCV(pipe_model, param_grid=hypermeter, cv=3, scoring='f1')  # 75*3 => 225번 학습이 수행
grid_model3.fit(X_train,Y_train)

C:\Users\UserK\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\UserK\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\UserK\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\UserK\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 345, in _score
    y_pred = method_caller(
             ^^^^^^^^^^^^^^
  File "C:\Users\UserK\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 87, in _

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('simpleimputer',
                                        SimpleImputer(strategy='median')),
                                       ('standardscaler', StandardScaler()),
                                       ('decisiontreeclassifier',
                                        DecisionTreeClassifier())]),
             param_grid={'decisiontreeclassifier__max_depth': range(5, 20),
                         'decisiontreeclassifier__min_samples_split': range(5, 10)},
             scoring='f1')

In [64]:
# Scikit Learn 라이브러리는 CPU를 이용해 연산을 수행
grid_model4 = GridSearchCV(pipe_model, param_grid=hypermeter, cv=3, scoring='f1',n_jobs=-1)  # n_jobs = -1 : CPU가 병렬로 처리하도록
grid_model4.fit(X_train,Y_train)

C:\Users\UserK\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:1051: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan]
  warnings.warn(


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('simpleimputer',
                                        SimpleImputer(strategy='median')),
                                       ('standardscaler', StandardScaler()),
                                       ('decisiontreeclassifier',
                                        DecisionTreeClassifier())]),
             n_jobs=-1,
             param_grid={'decisiontreeclassifier__max_depth': range(5, 20),
                         'decisiontreeclassifier__min_samples_split': range(5, 10)},
             scoring='f1')

In [68]:
 grid_model3.best_estimator_ 

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('standardscaler', StandardScaler()),
                ('decisiontreeclassifier',
                 DecisionTreeClassifier(max_depth=5, min_samples_split=5))])

In [65]:
best_model = grid_model3.best_estimator_ # 모든 경우의 수 중 가장 성능이 좋은 모델 도출
pickle.dump(best_model, open('model_cell.sav','wb')) # 가장 성능이 좋은 모델을 저장

7. 앞서 생성한 모델의 아래의 지표를 이용해 성능 평가를 수행하시오.
   - 
학습 능력 (Acc / Recall / Precision / F1
   - 
일반화 능력 (Acc / Recall / Precision / F1)

In [66]:
from sklearn.metrics import classification_report

In [67]:
Y_train_pred = best_model.predict(X_train)
Y_test_pred = best_model.predict(X_test)

In [69]:
print(classification_report(Y_train,Y_train_pred))
print(classification_report(Y_test,Y_test_pred))

              precision    recall  f1-score   support

           B       1.00      0.99      0.99       269
           M       0.98      0.99      0.99       157

    accuracy                           0.99       426
   macro avg       0.99      0.99      0.99       426
weighted avg       0.99      0.99      0.99       426

              precision    recall  f1-score   support

           B       0.90      0.95      0.93        88
           M       0.92      0.84      0.88        55

    accuracy                           0.91       143
   macro avg       0.91      0.90      0.90       143
weighted avg       0.91      0.91      0.91       143

